In [ ]:
#Imported libraries for my model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.cluster import KMeans

#Load the Zillow dataset
df = pd.read_csv("/home/jcn95/cs439/Final Project/cleaned_df.csv")

print("Original dataset of 2023 United States House Listings on Zillow:","\n\n", df.head())

#Data cleaning the cleaned_df.csv and creating features
df.dropna(subset=['ListedPrice', 'MarketEstimate', 'Area', 'Bedroom', 'Bathroom'], inplace=True)
df['Price Difference'] = df['ListedPrice']-df['MarketEstimate']
df['Overpriced'] = df['Price Difference'] > 20000
df['Price per Square Ft'] = df['ListedPrice']/df['Area']
df['Zip'] = df['Zipcode'].astype(str)

#Declare and initialize the features
house_features = ['Area', 'Bedroom', 'Bathroom', 'LotArea', 'RentEstimate', 'Price per Square Ft', 'Zip']

obj = 'MarketEstimate'

#Standard scaler features
X = df[house_features].dropna()
y = df.loc[X.index, obj]

house_scaler = StandardScaler()
hscaled = house_scaler.fit_transform(X)

#Training and testing
X_train, X_test, y_train, y_test = train_test_split(hscaled, y, test_size=0.5)

#Gradient Boosting Regressor model
houselist_model = GradientBoostingRegressor()
houselist_model.fit(X_train, y_train)
prediction = houselist_model.predict(X_test)

#Evaluation of my model and output
mae = mean_absolute_error(y_test, prediction)
r2 = r2_score(y_test, prediction)

print("\n\n" f"Mean Absolute Error of 2023 U.S. House Listings: ${mae:,.2f}")
print(f"R^2 Score of 2023 U.S. House Listings: ${r2:,.3f}")

#Prediction and labeling of listing
df_p = df.dropna(subset=house_features).copy()
df_s = house_scaler.transform(df_p[house_features])

df_p['Predicted Listing'] = houselist_model.predict(df_s)

df_p['Type of Pricing'] = np.where(df_p['Predicted Listing'] < df_p['ListedPrice'] - 20000, 'Overpriced', np.where(df_p['Predicted Listing'] > df_p['ListedPrice'] + 20000, 'Underpriced', 'Fair'))

#Output of predictions
print(df_p[['ListedPrice', 'Predicted Listing', 'Type of Pricing']].head())

#SQL database created from cleaned_df.db
house_sql = sqlite3.connect("/home/jcn95/cs439/Final Project/cleaned_df.db")
df_p['Zipcode_cleaned'] = df_p['Zipcode'].astype(str).str.zfill(5)

#SQL schema 
sql_dtype = {
    'Zipcode': 'TEXT'
}

df_sqlp = df_p[['ListedPrice', 'MarketEstimate', 'Area', 'Bedroom', 'Bathroom', 'Zipcode_cleaned']].rename(columns={'Zipcode_cleaned': 'Zipcode'})

df_sqlp.to_sql("listings", house_sql, if_exists="replace", index=False, dtype=sql_dtype)

#SQL queries
house_query = "SELECT Zipcode, AVG(ListedPrice) as AvgPrice FROM listings GROUP BY Zipcode"
avg_zip = pd.read_sql_query(house_query, house_sql)
print("\nAverage Listed Price within Zipcode:\n", avg_zip.head())

#Visualization of price difference
plt.figure(figsize=(15, 8))
sns.histplot(df['Price Difference'], bins=50, kde=True)
plt.xlim(-20000, 20000)
plt.title("Listed and Market Estimate Price Difference")
plt.xlabel("Price Difference")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

#Output csv for user
df_p.to_csv("Listed_House_Model_Output.csv", index=False)

#KMeans cluster
cluster_house = df_p[['Area', 'Bedroom', 'Bathroom', 'Price per Square Ft']]
scluster = StandardScaler().fit_transform(cluster_house)

kmeans = KMeans(n_clusters=4, random_state=40)
df_p['Cluster'] = kmeans.fit_predict(scluster)

#Visualization of price distribution
sns.boxplot(data=df_p, x='Cluster', y='ListedPrice')
plt.title("Cluster of Price Listings")
plt.show()

#Visualization of price listings grouped by zipcode and outputs top 20 highest
avg_zip_price = df_p.groupby('Zipcode')['ListedPrice'].mean().sort_values()
highest_zip_avg = avg_zip_price.sort_values(ascending=False).head(20).reset_index()

plt.figure(figsize=(12,6))
sns.barplot(data=highest_zip_avg, x="Zipcode", y= "ListedPrice", palette="Blues_d")
plt.title("20 Zip Codes with Highest Average Listed Price")
plt.xticks(rotation=45)
plt.xlabel("Zipcode")
plt.ylabel("Average Listed Price (100k)")
plt.tight_layout()
plt.show()